# Paper histograms

In [1]:
from coffea import util
from coffea import hist
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.image as mpimg

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/usr/local/lib/python3.8/dist-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
p_types = ['eplus','gamma','piplus']
rbm_paths= ['/home/blazerjia/Publication/Saved_samples/RBM/eplus/wandb/run-20230526_005306-36kf9r7d',
            "/home/blazerjia/Publication/Saved_samples/RBM/gamma/wandb/run-20240706_182429-ilnnubof",
           "/home/blazerjia/Publication/Saved_samples/RBM/piplus/wandb/run-20230526_000511-li89mnsl"]

p_paths = ['/home/blazerjia/Publication/Saved_samples/DWAVE/eplus/wandb/run-20230526_003448-zonp856v',
            "/home/blazerjia/Publication/Saved_samples/DWAVE/gamma/wandb/run-20230530_014613-zqkv3juf",
           "/home/blazerjia/Publication/Saved_samples/DWAVE/piplus/wandb/run-20230526_001933-j9xaqijz"]

In [3]:
_COLOR_DICT = {"eplus":"c", "piplus":"y", "gamma":"b"}

In [4]:


_SCALES = ["linear"]

In [5]:
def plot_single_hist(c_hist, xscale, ax, p_type, layerhist=False, layernum=-1, synthetic=True, doublebin = 0):
#         ax.set_xlim(0.2, 1)
        """
        Args:
            c_hist - coffea hist object
            xscale - xscale of the histogram (log, linear, ...)
            ax - Axes object to plot on
        Returns:
            image - PIL image object
        """
        assert len(c_hist.axes()) == 2, "Histogram should only have two axes - Dataset type and Bins"
        ax_0, ax_1 = c_hist.axes()[0], c_hist.axes()[1]
        
        if isinstance(ax_0, hist.Cat) and isinstance(ax_1, hist.Bin):
            cat_ax = ax_0
            bin_ax = ax_1
        elif isinstance(ax_0, hist.Bin) and isinstance(ax_0, hist.Cat):
            bin_ax = ax_0
            cat_ax = ax_1
        else:
            raise ValueError("Expected categorical and bin axis")
        
        cat_names = [identifier.name for identifier in cat_ax.identifiers()]
        bins = [ax_bin.hi for ax_bin in bin_ax.identifiers()]
        
        value_dict = {cat_name:c_hist.values(overflow='all')[(cat_name,)] for cat_name in cat_names}
      
        if xscale == "log":
            hi_bins = [ax_bin.hi for ax_bin in bin_ax.identifiers()]
            lo_bins = [ax_bin.lo for ax_bin in bin_ax.identifiers()]
            diff_bins = [hi_bin - lo_bin for hi_bin, lo_bin in zip(hi_bins, lo_bins)]
        else:
            hi_bins = [ax_bin.hi for ax_bin in bin_ax.identifiers()]
            lo_bins = [ax_bin.lo for ax_bin in bin_ax.identifiers()]
            diff_bins = [hi_bin - lo_bin for hi_bin, lo_bin in zip(hi_bins, lo_bins)][0]
        

        bins = [bins[0]] + bins

        for cat_name in cat_names:
            if cat_name == "input" or cat_name == "samples" or cat_name == "true":
                value_dict[cat_name]=value_dict[cat_name][:-1]
                qvae_size=sum(value_dict[cat_name])
                if xscale == "log":
                    scaled_value_dict = value_dict[cat_name][1:]/(qvae_size*np.array(diff_bins))
                    scaled_value_dict = np.insert(scaled_value_dict, 0, 0)
                else:
                    scaled_value_dict = value_dict[cat_name]/(qvae_size*diff_bins)
                    
                scaled_value_dict[0] = 0.
                scaled_value_dict[-1] = 0.
                if doublebin == 1:
                    if len(bins) % 2 == 1:
                        updated_bins = [bins[i] for i in range(0, len(bins)-1, 2)]
                    else:
                        updated_bins = [bins[i] for i in range(0, len(bins), 2)]
                    # 处理scaled_value_dict
                    updated_scaled_values = [scaled_value_dict[i] + scaled_value_dict[i+1] for i in range(0, len(scaled_value_dict)-1, 2)]
                    # updated_scaled_values[0] = 0.
                    # updated_scaled_values[-1] = 0.

                elif doublebin == 2:
                    if len(bins) % 4 == 1:
                        updated_bins = [bins[i] for i in range(0, len(bins)-1, 4)]
                    else:
                        updated_bins = [bins[i] for i in range(0, len(bins), 4)]
                    # 处理scaled_value_dict
                    updated_scaled_values = [scaled_value_dict[i] + scaled_value_dict[i+1]+ scaled_value_dict[i+2]+ scaled_value_dict[i+3] for i in range(0, len(scaled_value_dict)-1, 4)]
                    # updated_scaled_values[0] = 0.
                    updated_scaled_values[-1] = 0. 
                else:
                    updated_bins = bins
                    updated_scaled_values = scaled_value_dict 
                    # updated_scaled_values[0] = 0.
                    # updated_scaled_values[-1] = 0.  
                updated_scaled_values = np.insert(updated_scaled_values, 0, 0)
                updated_scaled_values = np.insert(updated_scaled_values, -1, 0)
                updated_bins = [-0.0001] + updated_bins
                updated_bins = updated_bins + [updated_bins[-1] + 0.0001]           
                    # if len(scaled_value_dict) % 2 == 1:
                #     updated_scaled_values.append(scaled_value_dict[-1])
                if p_type == "eplus":
                    p_label = r"""$e^{+}$"""
                elif p_type == "gamma":
                    p_label = r"""$\gamma$  """
                else:
                    p_label = r"""$\pi^{+}$"""   
                if cat_name == "input":
                    # label=p_label + " " + "QVAE"
                    if synthetic == True:
                        # ax.fill_between(updated_bins, updated_scaled_values, label="GEANT", step="post", color=(1, 0, 0, 0), hatch='/', edgecolor ='black')
                        ax.fill_between(updated_bins, updated_scaled_values, label="GEANT", step="post", color='black', alpha = 0.2,  edgecolor ='black')
                else:
                    line_width = 12.0
                    if synthetic == True:
                        ax.step(updated_bins, updated_scaled_values, label="QVAE", where="post", color='black', linewidth=line_width)
                    else:
                        ax.step(updated_bins, updated_scaled_values, label="DVAE", linestyle='dashed', where="post", color='black', linewidth=line_width)
                # else:
                #     if synthetic == True:
                #         ax.step(updated_bins, updated_scaled_values, label="QVAE", where="post", color='red', linewidth=12.0)
                #     else:
                #         ax.step(updated_bins, updated_scaled_values, label="DVAE",  where="post", color='yellow', linewidth=12.0)
            
        # if layerhist:
        #     xlabel = "Layer " + str(layernum) + " {0}".format(bin_ax.label)
        # else:
        #     xlabel = bin_ax.label
            
        # ax.set_xlabel(xlabel, fontsize='120')
        # ax.set_xlim(0,1)
        for spine in ax.spines.values():
            spine.set_color('black') 
            spine.set_linewidth(10)
        
        if xscale == "log":
            ax.set_ylabel(c_hist.label + " / Bin Width", fontsize='120')
            ax.set_xlim(1, max(updated_bins))
        # else:
        #     ax.set_ylabel('Density', fontsize='140')
            
        return

In [6]:
def plot_hist(ax, p_types, p_paths, rbm_paths, h_type, yscale, xscale="linear", legend_loc=None, legend_col=1, doublebin = 0):
    hist_dict = {}
    for p_type, p_path in zip(p_types, p_paths):
        hist_dict[p_type] = util.load(os.path.join(p_path, "files/{0}_{1}.coffea".format(p_type, h_type)))
    
    for p_type in p_types:
        single_hist = hist_dict[p_type]
        
        if "layer" in h_type:
            layerhist = True
            layer = int(h_type[6:7])
            plot_single_hist(single_hist, xscale, ax, p_type, layerhist=True, layernum=layer, synthetic=True, doublebin = doublebin)
        else:
            plot_single_hist(single_hist, xscale, ax, p_type, synthetic=True, doublebin = doublebin)
            
    hist_dict = {}
    for p_type, rbm_path in zip(p_types, rbm_paths):
        hist_dict[p_type] = util.load(os.path.join(rbm_path, "files/{0}_{1}.coffea".format(p_type, h_type)))
    
    for p_type in p_types:
        single_hist = hist_dict[p_type]
        
        if "layer" in h_type:
            layerhist = True
            layer = int(h_type[6:7])
            plot_single_hist(single_hist, xscale, ax, p_type, layerhist=True, layernum=layer, synthetic=False, doublebin = doublebin)
        else:
            plot_single_hist(single_hist, xscale, ax, p_type, synthetic=False, doublebin = doublebin)
            
    handles, labels = ax.get_legend_handles_labels()
    order = [0,1,2]
    ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order],prop={'size': 110}, loc=legend_loc, ncol=legend_col, framealpha=0) 
    #ax.legend(prop={'size': 40}, loc=legend_loc, ncol=legend_col, framealpha=0.3)
    ax.tick_params(axis='both', which='major', labelsize=130)
    ax.set_xscale(xscale)
    ax.set_yscale(yscale)
    if h_type == "dwTotalEnergyHist":
        ax.set_yticks([10**-5,10**-4,10**-3,10**-2,10**-1, 10**0, 10**0.3])
        ax.set_yticklabels(['','$10^{-4}$','$10^{-3}$','$10^{-2}$','$10^{-1}$','$10^0$',''])
        ax.set_xticks([10**-2.4,10**-2,10**-1, 10**0, 10**1, 10**2, 10**2.4])
        ax.set_xticklabels(['','$10^{-2}$','$10^{-1}$', '$10^0$', '$10^1$', '$10^2$',''])
    if h_type == "totalEnergyHist":
        ax.set_yticks([10**-4,10**-3,10**-2,10**-1, 10**0, 10**0.3])
        ax.set_ylim(10**(-4.2), 1e0)
        ax.set_yticklabels(['$10^{-4}$','$10^{-3}$','$10^{-2}$','$10^{-1}$','$10^0$',''])
        # ax.set_xticks([10**-2.4,10**-2,10**-1, 10**0, 10**1, 10**2, 10**2.4])
        # ax.set_xticklabels(['','$10^{-2}$','$10^{-1}$', '$10^0$', '$10^1$', '$10^2$',''])

    if h_type == "layer_1_fracEnergyHist":
        ax.set_yticks([10**-3, 10**-2, 10**-1, 10**0, 10**1])
        ax.set_yticklabels(['','$10^{-2}$','$10^{-1}$', '$10^0$', '$10^1$'])
    
    if h_type == "layer_1_sparsityHist":
        ax.set_yticks([10**-2.3, 10**-2, 10**-1, 10**0, 10**1])
        ax.set_yticklabels(['','$10^{-2}$','$10^{-1}$', '$10^0$', '$10^1$'])
        # ax.set_xticks([1, 10**1, 10**2])
        # ax.set_xticklabels(['$1$', '$10^1$', '$10^2$'])
    # ax.set_yticklabels(['','$10^{-2}$', '', '','$10^1$'])
    # if yscale == "log" and h_type == "dwTotalEnergyHist":
    #     ax.set_ylim(1e0, 1e2)

In [11]:
_HISTOGRAMS = ["totalEnergyHist", "layer_0_EnergyHist", "layer_1_EnergyHist", "layer_2_EnergyHist",
               "layer_0_sparsityHist", "layer_1_sparsityHist", "layer_2_sparsityHist",
               "layer_0_fracEnergyHist", "layer_1_fracEnergyHist", "layer_2_fracEnergyHist",
               "showerDepthHist", "sampleEnergyHist", "dwTotalEnergyHist"]


In [12]:
plot_all_hists_paper(savefig=True)